#### Importing Libraries

In [1]:
from os import path
import mlrun

#### Initializing

In [2]:
# Set the base project name
project_name_base = 'securenet'
# Initialize the MLRun environment and save the project name and artifacts path
project_name, artifact_path = mlrun.set_environment(project=project_name_base,
                                                    user_project=True)

#### Serving class

In [3]:
# nuclio: start-code

In [4]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [5]:
# nuclio: end-code

#### Converting serving class to a serving function

In [6]:
from mlrun import code_to_function
serving_fn = code_to_function('serving', kind='serving',image='mlrun/mlrun')
serving_fn.spec.default_class = 'ClassifierModel'

In [9]:
model_file = f'store://artifacts/securenet-floyed/train-data-train_data_model:latest'
modeldata = mlrun.get_dataitem(model_file)
modeldata



'v3io:///projects/securenet-floyed/artifacts/models/'

In [10]:
serving_fn.add_model('lr_model',model_path=model_file)

In [11]:
from mlrun.platforms import auto_mount
serving_fn = serving_fn.apply(auto_mount())

In [12]:
project_path = path.abspath('conf')
securenet_proj =mlrun.new_project(project_name_base,
                            context=project_path,
                            init_git=True,
                            user_project=True)

In [13]:

securenet_proj.set_function(serving_fn)

In [14]:
securenet_proj.save()

In [15]:
from mlrun.platforms import auto_mount
serving_fn = serving_fn.apply(auto_mount())

## Testing locally

In [16]:
df = mlrun.get_dataitem('store://artifacts/securenet-floyed/data_clean_cleaned_data:latest').as_df()

df = df.tail()
df

,url,url_len,ip_add,geo_loc,tld,who_is,https,label,special_chars_count,parameter_count,digit_count
53858,22747,50,23996,175,64,1,0,0,8,0,0
53859,53541,37,12735,153,64,0,0,0,8,0,0
53860,40963,42,1483,112,64,1,0,0,8,0,0
53861,28222,26,15549,56,282,1,0,0,7,0,0
53862,34657,27,43318,87,64,1,0,0,7,0,0


In [ ]:
print(len(df))

In [19]:
import json
my_data = df.values.tolist()

In [25]:
my_data = '''{"inputs":[[22747, 50, 23996, 175,64,1,0,8,0]]}'''

In [26]:
server = serving_fn.to_mock_server()
server.test("/v2/models/lr_model/infer", body=my_data)

> 2021-07-06 06:26:06,362 [info] model lr_model was loaded
> 2021-07-06 06:26:06,363 [info] Loaded ['lr_model']


Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.


{'id': '22ad9ab35fbd416c8ddbf67ec683b737',
 'model_name': 'lr_model',
 'outputs': [1]}

In [29]:
function_address = serving_fn.deploy()

> 2021-07-06 06:27:08,537 [info] Starting remote function deploy
2021-07-06 06:27:08  (info) Deploying function
2021-07-06 06:27:08  (info) Building
2021-07-06 06:27:08  (info) Staging files and preparing base images
2021-07-06 06:27:09  (info) Building processor image
2021-07-06 06:27:11  (info) Build complete
2021-07-06 06:27:19  (info) Function deploy complete
> 2021-07-06 06:27:20,448 [info] function deployed, address=default-tenant.app.mlops9.iguazio-c0.com:32108


## Using model serving function

In [30]:
print (f'The address for the function is {function_address} \n')

!curl $function_address

The address for the function is http://default-tenant.app.mlops9.iguazio-c0.com:32108 

{"name": "ModelRouter", "version": "v2", "extensions": []}

## Testing the model server

In [31]:
serving_fn.invoke('/v2/models/lr_model/infer', my_data)

{'id': 'b19f64cf-1e70-4890-a4c0-8710164617b1',
 'model_name': 'lr_model',
 'outputs': [1]}